In [32]:
from inspect import BoundArguments
import sys
import mip 
from collections import deque

In [74]:


def leitura(file):
    f = open(file,'r')
    res_var = f.readline()
    nb_var,nb_res= int(res_var.split()[0]), int(res_var.split()[1])
    
    l = f.readline()
    max_objetivo = [float(m) for m in l.split()]

    reticoes = []
    for i in range(nb_var):
        l = f.readline()
        levels = [float(level) for level in l.split()]
        reticoes.append(levels)

    f.close()

    return nb_var,nb_res, costs, max_objetivo, reticoes

    
t= leitura("exemplo1.txt")
print(t)
    
    

NameError: name 'nb_foods' is not defined